In [1]:
from __future__ import division, print_function, absolute_import
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import json
import time
import numpy as np
import collections

import tensorflow as tf



import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import time

import gc
import matplotlib.pyplot as plt

/afs/inf.ed.ac.uk/user/s16/s1688201/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/afs/inf.ed.ac.uk/user/s16/s1688201/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/afs/inf.ed.ac.uk/user/s16/s1688201/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [2]:
class user(object):
    """docstring for User"""

    def __init__(self, userid):
        self.userid = userid

        self.polls = {}  # Key is pollId value is [interaction, index]

    def get_engagement_array(self, length):
        arr = np.zeros(length)
        for i in self.polls.keys():

            interaction = self.polls[i][0]

            idx = self.polls[i][1]
            arr[idx] = interaction
        return arr


    def add_interaction(self, pollid, interaction, pollidx):
        if pollid in self.polls.keys():

            if self.polls[pollid][0]>=0 and interaction>=0:
                self.polls[pollid][0] += interaction

            if self.polls[pollid][0]>=0 and interaction<0:
                self.polls[pollid][0] = self.polls[pollid][0]

            if self.polls[pollid][0] < 0 and interaction >= 0:
                self.polls[pollid][0] = interaction

            if self.polls[pollid][0] < 0 and interaction < 0:
                self.polls[pollid][0] = interaction
        else:
            self.polls[pollid] = [interaction, pollidx]

    def num_interactions(self):
        return len(self.polls)

In [3]:
class poll(object):
    """docstring for Poll"""

    def __init__(self, userid, pollid, title, this, that, boost=0):
        self.pollid = pollid
        self.userid = userid
        self.title = title
        self.this = this
        self.that = that
        self.boost = boost

    def get_text(self):
        return self.title + ' ' + self.this + '' + self.that

In [4]:
class data_preprocessing(object):

    def __init__(self, json_location= '/afs/inf.ed.ac.uk/user/s16/s1688201/Desktop/my_honours_project/failed model/this_that.json'):
        self.json_location = json_location
        self.polls = []
        self.users = collections.OrderedDict()
        self.data = []
        self.train = []
        self.test = []
        self.validation = []
        self.train_proportion = 0.70
        self.validation_proportion = 0.15


    def read_json(self):
        with open(self.json_location, 'r') as file:
            content = file.read()
        self.data = json.loads(content)

    def put_users_into_user_list(self):
        keys = self.data['users'].keys()
        for i in set(keys):
            self.users[i] = user(i)

    def put_polls_into_poll_list_and_add_user_interaction(self):
        for i in self.data['polls']:

            for j in self.data['polls'][i]:

                title = self.data['polls'][i][j]['question']
                this = self.data['polls'][i][j]['this']
                that = self.data['polls'][i][j]['that']

                p = poll(self.users[i].userid, j, title, this, that)
                self.polls.append(p)

                self.users[i].add_interaction(j, 5, self.polls.index(p))

    def check_which_user_skip_or_vote_on_the_poll(self):
        for i in self.data['votes'].keys():  # I = pollid
            if next((x for x in self.polls if x.pollid == i), None) == None:
                continue
            pollidx = self.polls.index(next((x for x in self.polls if x.pollid == i), None))
            for j in self.data['votes'][i].keys():  # J is userid
                if j not in self.users.keys():
                    self.users[j] = user(j)
                # this is where the user skips the poll
                if j in self.users.keys() and u'skip' in self.data['votes'][i][j].keys():
                    self.users[j].add_interaction(i, -1, pollidx)
                # this is where the user votes on the poll
                if j in self.users.keys() and u'this0that1removed2' in self.data['votes'][i][j].keys() and \
                        self.data['votes'][i][j][u'this0that1removed2'] in [0, 1]:
                    self.users[j].add_interaction(i, 2, pollidx)

    def check_which_user_comment_on_the_poll(self):
        for i in self.data['commentsPath'].keys():  # i = userID
            for j in self.data['commentsPath'][i].keys():  # j is pollId
                if next((x for x in self.polls if x.pollid == j), None) == None:
                    continue
                pollidx = self.polls.index(next((x for x in self.polls if x.pollid == j), None))
                # this is where the user comment on the poll
                self.users[i].add_interaction(j, 3, pollidx)

    def check_which_user_track_on_the_poll(self):
        for i in self.data['tracking'].keys():  # i = userID
            for j in self.data['tracking'][i].keys():  # j is unknown
                for k in self.data['tracking'][i][j].keys():
                    if next((x for x in self.polls if x.pollid == k), None) == None:
                        continue
                    pollidx = self.polls.index(next((x for x in self.polls if x.pollid == k), None))
                    if i not in self.users.keys():
                        self.users[i] = user(i)
                        # this is where the user track on the poll
                    self.users[i].add_interaction(k, 4, pollidx)

    def set_initial_training_and_test_data(self):
        # indices = np.random.choice(range(len(self.users)), int(len(self.users)*self.train_proportion), replace=False)

        training_amount = int(len(self.users) * self.train_proportion)
        validation_amount = int(len(self.users) * self.validation_proportion)

        training_indices = np.arange(0, training_amount)
        validation_indices = np.arange(training_amount, training_amount + validation_amount)

        total_polls = len(self.polls)
        for idx, i in enumerate(self.users.keys()):
            if self.users[i].num_interactions() > 40:
                poll_array = self.users[i].get_engagement_array(total_polls)

                if idx in training_indices:
                    self.train.append(poll_array)
                if idx in validation_indices:
                    self.validation.append(poll_array)
                else:
                    self.test.append(poll_array)


        self.train = np.asarray(self.train)
        self.test = np.asarray(self.test)
        self.validation = np.asarray(self.validation)

    def parse(self):

        self.read_json()

        self.put_users_into_user_list()

        self.put_polls_into_poll_list_and_add_user_interaction()

        self.check_which_user_skip_or_vote_on_the_poll()

        self.check_which_user_comment_on_the_poll()

        self.check_which_user_track_on_the_poll()

        self.set_initial_training_and_test_data()

In [ ]:
'''
Step 1:
Pick up an Embedding Vector from User Embedding Matrix for a particular user 
and an Embedding Vector from Poll Embedding Matrix for a particular Poll.

Step 2 : 
Concatenate both of the vector. 
Do the same for all the movies and Users, that we want.
It will lead to a Embedding matrix where each row is made up of concatenation of Polls and User Embedding vectors.
Number of rows will be the same as Number of Users and Polls combination. 
And the column is decided by the number of factors. 
Same as Embedding dimensionality, that we discussed before. 
So the dimensions that we have now is ( # ( Users + Polls ) combination , #Factors )
'''



'''
Step 3 :
Feed it to a 1st Linear Layer where the dimension of the Linear Layer is (#Factors,10). The output will be (#(Users+Movies)combination,10).

Step 4:
Pass it through a Second Linear Layer with ReLU activation having a dimension of (10,1). The output will be (#(Users+Movies)combination,1).
'''


'''
Step 5:
This is the output that we wanted.
Earlier what we did is for each User and Movie, we did a cross product of their Embeddings which gave us a single number i.e Predicted Rating. 
Here we are using Neural Network to get that single number i.e Predicted Rating.

Step 6:
We will then try to minimize the Loss between Predicted Rating and Actual Rating, and in turn will update the model parameters (Embedding values). 
In this way we can make our predictions more accurate.
'''
class neural_network(object):# 155 27 128 101 74
    def __init__(self,num_epochs=10000, display_step=100, learning_rate=0.001, batch_size=100,
                 denoising=False, new_poll_weight=0.002,masking=0, num_layers=3, num_hidden_1=10,
                 num_hidden_2=81,num_hidden_3=27,num_hidden_4=9,num_hidden_5=3,continue_from_saved=False, time_decay=1):

        self.X = tf.placeholder("float",[None,None])
        self.Y = tf.placeholder("float", [None, None])
        self.num_hidden_1 = num_hidden_1
        self.data_provider =data_preprocessing()
        self.data_provider.parse()
        # Interactions are fed as binary but using decimal helps with adding 2 interactions together


        self.num_epochs = num_epochs # initial training eppchs given training data

        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.denoising = denoising # whether to add noise to the input vectors - might help with accidental interactions
        self.new_poll_weight = new_poll_weight # How much weight new polls are given in the output layer (gives new polls some initial traction)
        self.masking = masking # TODO: Add masking to data to make synthetic users with less interactions and see if it helps
        self.num_layers = num_layers



        self.test_users = []
        self.time_decay = time_decay

        self.saver = None
        self.compressed_train = []
        self.compressed_test = []

        self.factor = int(10)
        self.num_hidden_1 = 10
        self.num_hidden_2 = 1

        self.training_data = self.data_provider.train
        self.testing_data = self.data_provider.test
        validation_data = self.data_provider.validation

        (num_of_users, num_of_polls) = np.shape(self.training_data)
      

        self.training_data_x = np.random.normal(size=((num_of_users * num_of_polls), self.factor * 2))
        training_data_y = np.asarray(self.training_data)
        self.training_data_y = np.reshape(training_data_y, (-1, 1))

        (num_of_validation_users, num_of_validation_polls) = np.shape(validation_data)
        self.validation_data_x = np.random.normal(size=((num_of_validation_users * num_of_validation_polls), self.factor * 2))
        validation_data_y = np.asarray(validation_data)
        self.validation_data_y = np.reshape(validation_data_y, (1, -1))

        self.weights = {
            'layer_1': tf.Variable(tf.random.normal([self.factor*2, self.num_hidden_1])),
            'layer_2': tf.Variable(tf.random.normal([self.num_hidden_1, self.num_hidden_2])),
        }

        self.bias = {
            'bias_1': tf.Variable(tf.random.normal([self.num_hidden_1])),
            'bias_2': tf.Variable(tf.random.normal([self.num_hidden_2])),
        }


        self.encoder_op = self.forward(self.X)

        self.setup_graph()

        self.train_and_predict()

    def forward(self, x):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, self.weights['layer_1']), self.bias['bias_1']))
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, self.weights['layer_2']), self.bias['bias_2']))
        return layer_2

    def setup_graph(self):
        tf.global_variables_initializer()
        # Prediction
        self.y_pred = self.encoder_op

        self.y_true = self.Y
        self.loss = self._loss()
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)

        self.init = tf.global_variables_initializer()
        self.saver = tf.train.Saver()

    def _loss(self):
        return tf.reduce_mean(tf.pow(self.y_true - self.y_pred, 2))

    def train_and_predict(self,save=True):
        gc.collect()

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            train_loss = []
            validation_loss = []

            for i in range(0, self.num_epochs):
                np.random.seed(0)
                batch_x = self.training_data_x[np.random.choice(self.training_data_x.shape[0], self.batch_size, replace=True), :]
                np.random.seed(0)


                batch_y = self.training_data_y[np.random.choice(self.training_data_x.shape[0], self.batch_size, replace=True), :]

                _, l_train = sess.run([self.optimizer, self.loss], feed_dict={self.X: batch_x, self.Y: batch_y})
                train_loss.append(l_train)

                l_validation = sess.run(self.loss, feed_dict={self.X: self.validation_data_x, self.Y: self.validation_data_y})
                validation_loss.append(l_validation)

                print('num of epoch is %f and the training error is %f and validation error is %f'%(i,l_train,l_validation))


            xs = np.arange(1,self.num_epochs+1,1)
            ys = train_loss
            fig, ax = plt.subplots()
            ax.plot(xs, ys,color = 'b')
            xs = np.arange(1, self.num_epochs+1, 1)
            ys = validation_loss
            plt.plot(xs, ys,color = 'r')
            plt.show()

            print('autoencoder finished')
        sess.close()

a  = neural_network()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
num of epoch is 0.000000 and the training error is 99.750740 and validation error is 74.553154
num of epoch is 1.000000 and the training error is 97.857597 and validation error is 73.547226
num of epoch is 2.000000 and the training error is 96.426514 and validation error is 72.695320
num of epoch is 3.000000 and the training error is 95.211258 and validation error is 71.936180
num of epoch is 4.000000 and the training error is 94.127754 and validation error is 71.239983
num of epoch is 5.000000 and the training error is 93.133224 and validation error is 70.589920
num of epoch is 6.000000 and the training error is 92.204063 and validation error is 69.975204
num of epoch is 7.000000 and the training error is 91.325058 and validation error is 69.388718
num of epoch is 8.000000 and

num of epoch is 78.000000 and the training error is 51.807938 and validation error is 42.089481
num of epoch is 79.000000 and the training error is 51.376644 and validation error is 41.787334
num of epoch is 80.000000 and the training error is 50.949051 and validation error is 41.487034
num of epoch is 81.000000 and the training error is 50.527519 and validation error is 41.187618
num of epoch is 82.000000 and the training error is 50.107544 and validation error is 40.890278
num of epoch is 83.000000 and the training error is 49.690907 and validation error is 40.594952
num of epoch is 84.000000 and the training error is 49.277428 and validation error is 40.301476
num of epoch is 85.000000 and the training error is 48.866955 and validation error is 40.009949
num of epoch is 86.000000 and the training error is 48.459404 and validation error is 39.720287
num of epoch is 87.000000 and the training error is 48.054688 and validation error is 39.432568
num of epoch is 88.000000 and the traini

num of epoch is 163.000000 and the training error is 23.741615 and validation error is 22.249287
num of epoch is 164.000000 and the training error is 23.499878 and validation error is 22.078173
num of epoch is 165.000000 and the training error is 23.260166 and validation error is 21.908396
num of epoch is 166.000000 and the training error is 23.022461 and validation error is 21.739933
num of epoch is 167.000000 and the training error is 22.786751 and validation error is 21.572815
num of epoch is 168.000000 and the training error is 22.553112 and validation error is 21.407166
num of epoch is 169.000000 and the training error is 22.321524 and validation error is 21.242905
num of epoch is 170.000000 and the training error is 22.091885 and validation error is 21.079872
num of epoch is 171.000000 and the training error is 21.864195 and validation error is 20.918093
num of epoch is 172.000000 and the training error is 21.638466 and validation error is 20.757627
num of epoch is 173.000000 and

num of epoch is 248.000000 and the training error is 9.311327 and validation error is 11.688705
num of epoch is 249.000000 and the training error is 9.194869 and validation error is 11.601654
num of epoch is 250.000000 and the training error is 9.080462 and validation error is 11.515880
num of epoch is 251.000000 and the training error is 8.968005 and validation error is 11.431314
num of epoch is 252.000000 and the training error is 8.858191 and validation error is 11.347382
num of epoch is 253.000000 and the training error is 8.750028 and validation error is 11.264743
num of epoch is 254.000000 and the training error is 8.643473 and validation error is 11.183153
num of epoch is 255.000000 and the training error is 8.538526 and validation error is 11.102220
num of epoch is 256.000000 and the training error is 8.435089 and validation error is 11.021847
num of epoch is 257.000000 and the training error is 8.333239 and validation error is 10.942691
num of epoch is 258.000000 and the train

num of epoch is 335.000000 and the training error is 3.485829 and validation error is 6.710993
num of epoch is 336.000000 and the training error is 3.453746 and validation error is 6.679018
num of epoch is 337.000000 and the training error is 3.422214 and validation error is 6.647580
num of epoch is 338.000000 and the training error is 3.391236 and validation error is 6.616579
num of epoch is 339.000000 and the training error is 3.360811 and validation error is 6.585966
num of epoch is 340.000000 and the training error is 3.330945 and validation error is 6.555858
num of epoch is 341.000000 and the training error is 3.301387 and validation error is 6.525443
num of epoch is 342.000000 and the training error is 3.271784 and validation error is 6.495389
num of epoch is 343.000000 and the training error is 3.242913 and validation error is 6.465741
num of epoch is 344.000000 and the training error is 3.214707 and validation error is 6.436354
num of epoch is 345.000000 and the training error 

...

num of epoch is 7652.000000 and the training error is 1.700067 and validation error is 3.966681
num of epoch is 7653.000000 and the training error is 1.700080 and validation error is 3.987030
num of epoch is 7654.000000 and the training error is 1.700104 and validation error is 3.960293
num of epoch is 7655.000000 and the training error is 1.700130 and validation error is 3.990929
num of epoch is 7656.000000 and the training error is 1.700137 and validation error is 3.960310
num of epoch is 7657.000000 and the training error is 1.700119 and validation error is 3.988061
num of epoch is 7658.000000 and the training error is 1.700094 and validation error is 3.963825
num of epoch is 7659.000000 and the training error is 1.700074 and validation error is 3.984924
num of epoch is 7660.000000 and the training error is 1.700062 and validation error is 3.966120
num of epoch is 7661.000000 and the training error is 1.700056 and validation error is 3.983250
num of epoch is 7662.000000 and the trai

num of epoch is 7738.000000 and the training error is 1.700069 and validation error is 3.981880
num of epoch is 7739.000000 and the training error is 1.700068 and validation error is 3.964839
num of epoch is 7740.000000 and the training error is 1.700072 and validation error is 3.987156
num of epoch is 7741.000000 and the training error is 1.700093 and validation error is 3.958062
num of epoch is 7742.000000 and the training error is 1.700146 and validation error is 3.993347
num of epoch is 7743.000000 and the training error is 1.700203 and validation error is 3.957333
num of epoch is 7744.000000 and the training error is 1.700188 and validation error is 3.987251
num of epoch is 7745.000000 and the training error is 1.700129 and validation error is 3.965362
num of epoch is 7746.000000 and the training error is 1.700082 and validation error is 3.980547
num of epoch is 7747.000000 and the training error is 1.700057 and validation error is 3.969899
num of epoch is 7748.000000 and the trai

num of epoch is 7824.000000 and the training error is 1.700043 and validation error is 3.983372
num of epoch is 7825.000000 and the training error is 1.700045 and validation error is 3.963324
num of epoch is 7826.000000 and the training error is 1.700055 and validation error is 3.985322
num of epoch is 7827.000000 and the training error is 1.700083 and validation error is 3.960892
num of epoch is 7828.000000 and the training error is 1.700146 and validation error is 3.986352
num of epoch is 7829.000000 and the training error is 1.700209 and validation error is 3.963478
num of epoch is 7830.000000 and the training error is 1.700194 and validation error is 3.980611
num of epoch is 7831.000000 and the training error is 1.700135 and validation error is 3.969155
num of epoch is 7832.000000 and the training error is 1.700085 and validation error is 3.976419
num of epoch is 7833.000000 and the training error is 1.700054 and validation error is 3.971763
num of epoch is 7834.000000 and the trai

num of epoch is 7910.000000 and the training error is 1.700135 and validation error is 3.962386
num of epoch is 7911.000000 and the training error is 1.700204 and validation error is 3.988416
num of epoch is 7912.000000 and the training error is 1.700220 and validation error is 3.958145
num of epoch is 7913.000000 and the training error is 1.700162 and validation error is 3.984948
num of epoch is 7914.000000 and the training error is 1.700094 and validation error is 3.963543
num of epoch is 7915.000000 and the training error is 1.700054 and validation error is 3.980584
num of epoch is 7916.000000 and the training error is 1.700035 and validation error is 3.966158
num of epoch is 7917.000000 and the training error is 1.700026 and validation error is 3.979348
num of epoch is 7918.000000 and the training error is 1.700024 and validation error is 3.966221
num of epoch is 7919.000000 and the training error is 1.700026 and validation error is 3.980314
num of epoch is 7920.000000 and the trai

num of epoch is 7996.000000 and the training error is 1.700074 and validation error is 3.963332
num of epoch is 7997.000000 and the training error is 1.700047 and validation error is 3.979953
num of epoch is 7998.000000 and the training error is 1.700034 and validation error is 3.965251
num of epoch is 7999.000000 and the training error is 1.700028 and validation error is 3.979264
num of epoch is 8000.000000 and the training error is 1.700028 and validation error is 3.964822
num of epoch is 8001.000000 and the training error is 1.700034 and validation error is 3.980718
num of epoch is 8002.000000 and the training error is 1.700046 and validation error is 3.962531
num of epoch is 8003.000000 and the training error is 1.700067 and validation error is 3.983249
num of epoch is 8004.000000 and the training error is 1.700092 and validation error is 3.960932
num of epoch is 8005.000000 and the training error is 1.700103 and validation error is 3.983020
num of epoch is 8006.000000 and the trai